# <font color=teal>imports</font>
Most processing is performed in python code, and there's a python module to do everything here without manual

In [1]:
import logging

from src import configs
import os
import sys
sys.path.append(os.path.abspath("../src"))



In [2]:
from src.nflverse_transform_job import load_files
from src.pbp_fact import transform_pbp
from src.nflverse_reader_job import URLReader
from src.pbp_participation import transform_pbp_participation
from src.player_stats import transform_player_stats, merge_injuries
from src.player_injuries import prep_player_injuries
from src.player_stats import transform_players
from src.db_utils import load_dims_to_db


# <font color=teal>housekeeping</font>

In [3]:
LOAD_TO_DB = True
database_schema = 'controls'

# Get the logger
logger = configs.configure_logging("pbp_logger")
logger.setLevel(logging.INFO)

# <font color=teal>read data from nflverse<font/>
Step 1.  Read data and store immediately as raw without transformation or change

In [4]:
%%time

reader = URLReader(start_year=2016, last_year=2022, file_type='csv')
urls = reader.download()

2023-07-06 18:11:42,038 - INFO - Success: https://github.com/nflverse/nflverse-data/releases/download/nextgen_stats/ngs_2016_receiving.csv.gz
2023-07-06 18:11:42,099 - INFO - Success: https://github.com/nflverse/nflverse-data/releases/download/nextgen_stats/ngs_2016_rushing.csv.gz
2023-07-06 18:11:42,372 - INFO - Success: https://github.com/nflverse/nflverse-data/releases/download/nextgen_stats/ngs_2016_passing.csv.gz
2023-07-06 18:11:42,623 - INFO - Success: https://github.com/nflverse/nflverse-data/releases/download/injuries/injuries_2016.csv
2023-07-06 18:11:42,856 - INFO - Success: https://github.com/nflverse/nflverse-data/releases/download/players/players.csv
2023-07-06 18:11:43,013 - INFO - Success: https://github.com/nflverse/nflverse-data/releases/download/nextgen_stats/ngs_2017_rushing.csv.gz
2023-07-06 18:11:43,151 - INFO - Success: https://github.com/nflverse/nflverse-data/releases/download/injuries/injuries_2017.csv
2023-07-06 18:11:43,234 - INFO - Success: https://github.c

CPU times: user 2.32 s, sys: 1.96 s, total: 4.27 s
Wall time: 18.9 s


---

# <font color=teal>load and transform play by play datasets</font>

### <font color="#9370DB">load</font>

In [5]:

%%time
pbp = load_files(data_subdir='pbp')


2023-07-06 18:12:06,526 - INFO - Reading all files from pbp
2023-07-06 18:12:06,527 - INFO -   + Reading pbp_2022.csv
2023-07-06 18:12:07,595 - INFO -   + Reading pbp_2020.csv
2023-07-06 18:12:08,596 - INFO -   + Reading pbp_2021.csv
2023-07-06 18:12:09,672 - INFO -   + Reading pbp_2019.csv
2023-07-06 18:12:10,645 - INFO -   + Reading pbp_2018.csv
2023-07-06 18:12:11,648 - INFO -   + Reading pbp_2016.csv
2023-07-06 18:12:12,652 - INFO -   + Reading pbp_2017.csv


CPU times: user 6.49 s, sys: 1.38 s, total: 7.88 s
Wall time: 8.34 s


### <font color="#9370DB">transform</font>

In [6]:

%%time
datasets = transform_pbp(pbp)

2023-07-06 18:12:18,552 - INFO - Impute columns to 0
2023-07-06 18:12:18,745 - INFO - impute non binary pbp columns ...
2023-07-06 18:12:19,218 - INFO - Impute columns to 0
2023-07-06 18:12:19,753 - INFO - Impute columns to 0:00
2023-07-06 18:12:20,847 - INFO - Impute columns to NA
2023-07-06 18:12:25,972 - INFO - moving play_id to play_counter, and creating a joinable play_id key
2023-07-06 18:12:26,499 - INFO - Conform key actions like pass, rush, kickoff, etc. and add a single category field called actions... 
2023-07-06 18:12:31,304 - INFO - Validate actions dimension ...
2023-07-06 18:12:31,476 - INFO - Creating new drive dimension...
2023-07-06 18:12:31,528 - INFO - Validate drive_df dimension ...
2023-07-06 18:12:31,757 - INFO - Creating new situations dimension...
2023-07-06 18:12:31,805 - INFO - Validate situation_df dimension ...
2023-07-06 18:12:31,999 - INFO - Creating new metrics dimension...
2023-07-06 18:12:32,038 - INFO - Validate play_metrics_df dimension ...
2023-07-0

CPU times: user 15 s, sys: 4.34 s, total: 19.3 s
Wall time: 20 s


---

# <font color=teal>load and transform play by play participation datasets</font>

### <font color="#9370DB">load</font>

In [7]:
%%time
pbp_participation_df = load_files('pbp-participation')


2023-07-06 18:12:36,953 - INFO - Reading all files from pbp-participation
2023-07-06 18:12:36,954 - INFO -   + Reading pbp-participation_2018.csv
2023-07-06 18:12:37,104 - INFO -   + Reading pbp-participation_2019.csv
2023-07-06 18:12:37,249 - INFO -   + Reading pbp-participation_2021.csv
2023-07-06 18:12:37,406 - INFO -   + Reading pbp-participation_2020.csv
2023-07-06 18:12:37,551 - INFO -   + Reading pbp-participation_2022.csv
2023-07-06 18:12:37,697 - INFO -   + Reading pbp-participation_2017.csv
2023-07-06 18:12:37,851 - INFO -   + Reading pbp-participation_2016.csv


CPU times: user 1.11 s, sys: 111 ms, total: 1.22 s
Wall time: 1.22 s


### <font color="#9370DB">transform</font>

In [8]:

player_df, player_events_df = transform_pbp_participation(
    participation_df=pbp_participation_df,
    player_events=datasets['player_events'])

datasets.update({
    'player_participation': player_df,
    'player_events': player_events_df,
})

2023-07-06 18:12:38,179 - INFO - pbp_participation:  move play_id to a play_count column and create a unique play_id that can be used in joins...
2023-07-06 18:12:38,398 - INFO - Calculating defense and offense team names by player and play...
2023-07-06 18:12:40,584 - INFO - Exploding offensive players to their own dataset...
2023-07-06 18:12:41,981 - INFO - Exploding defense_players to their own dataset...
2023-07-06 18:12:49,642 - WARNING - combining offense and defense players - counts are incorrect 6903461 != 6837089


---

# <font color=teal>transform player injuries<</font>

### <font color="#9370DB">load</font>

In [9]:
%%time
injuries_df = load_files('injuries')

2023-07-06 18:12:56,795 - INFO - Reading all files from injuries
2023-07-06 18:12:56,796 - INFO -   + Reading injuries_2016.csv
2023-07-06 18:12:56,807 - INFO -   + Reading injuries_2017.csv
2023-07-06 18:12:56,820 - INFO -   + Reading injuries_2020.csv
2023-07-06 18:12:56,829 - INFO -   + Reading injuries_2021.csv
2023-07-06 18:12:56,839 - INFO -   + Reading injuries_2022.csv
2023-07-06 18:12:56,852 - INFO -   + Reading injuries_2019.csv
2023-07-06 18:12:56,862 - INFO -   + Reading injuries_2018.csv


CPU times: user 71.4 ms, sys: 12.9 ms, total: 84.2 ms
Wall time: 85.2 ms


### <font color="#9370DB">transform</font>

In [10]:
%%time
injuries_df = prep_player_injuries(injuries_df)

2023-07-06 18:12:59,952 - INFO - Prep injury data...
2023-07-06 18:12:59,952 - INFO - Conforming names (e.g. gsis_id -> player_id)
2023-07-06 18:12:59,962 - INFO - Merge sparse injury columns
2023-07-06 18:12:59,966 - INFO - Get best values for null report_statuses...
2023-07-06 18:13:00,033 - INFO - check that all positions are correct...


CPU times: user 81.6 ms, sys: 4.86 ms, total: 86.5 ms
Wall time: 88.5 ms


---

# <font color=teal>transform player stats</font>

In [11]:
%%time
stats_df = load_files('player-stats')
stats_df = transform_player_stats(stats_df)
stats_df = merge_injuries(player_stats=stats_df, player_injuries=injuries_df)

2023-07-06 18:13:02,184 - INFO - Reading all files from player-stats
2023-07-06 18:13:02,185 - INFO -   + Reading player-stats.csv
2023-07-06 18:13:02,471 - INFO - fix specific player_stats: <function player_stats_fixes at 0x137615b80>..
2023-07-06 18:13:02,544 - INFO - replace empty position_groups with position info...
2023-07-06 18:13:02,554 - INFO - replace empty player_name with player_display_name info...
2023-07-06 18:13:02,565 - INFO - replace empty headshot_url with 'none'...
2023-07-06 18:13:02,574 - INFO - fillna(0) for all binary columns...
2023-07-06 18:13:02,574 - INFO - Impute columns to 0
2023-07-06 18:13:02,745 - WARNING - After merge player_stats count changed - went from 124081 to 124083


CPU times: user 485 ms, sys: 84.8 ms, total: 570 ms
Wall time: 566 ms


---

# <font color=teal>direct loads </font>

### <font color="#9370DB">adv stats</font>

In [12]:
%%time

advstats_def_df = load_files('advstats-season-def')
advstats_pass_df = load_files('advstats-season-pass')
advstats_rec_df = load_files('advstats-season-rec')
advstats_rush_df = load_files('advstats-season-rush')


2023-07-06 18:13:05,648 - INFO - Reading all files from advstats-season-def
2023-07-06 18:13:05,649 - INFO -   + Reading advstats-season-def.csv
2023-07-06 18:13:05,657 - INFO - Reading all files from advstats-season-pass
2023-07-06 18:13:05,658 - INFO -   + Reading advstats-season-pass.csv
2023-07-06 18:13:05,663 - INFO - Reading all files from advstats-season-rec
2023-07-06 18:13:05,664 - INFO -   + Reading advstats-season-rec.csv
2023-07-06 18:13:05,671 - INFO - Reading all files from advstats-season-rush
2023-07-06 18:13:05,672 - INFO -   + Reading advstats-season-rush.csv


CPU times: user 22.4 ms, sys: 5.98 ms, total: 28.4 ms
Wall time: 28.2 ms


### <font color="#9370DB">nextgen stats</font>

In [13]:
%%time
next_pass_df = load_files('nextgen-passing')


2023-07-06 18:13:07,423 - INFO - Reading all files from nextgen-passing
2023-07-06 18:13:07,425 - INFO -   + Reading nextgen-passing_2017.csv.gz
2023-07-06 18:13:07,431 - INFO -   + Reading nextgen-passing_2021.csv.gz
2023-07-06 18:13:07,436 - INFO -   + Reading nextgen-passing_2019.csv.gz
2023-07-06 18:13:07,441 - INFO -   + Reading nextgen-passing_2020.csv.gz
2023-07-06 18:13:07,446 - INFO -   + Reading nextgen-passing_2016.csv.gz
2023-07-06 18:13:07,453 - INFO -   + Reading nextgen-passing_2022.csv.gz
2023-07-06 18:13:07,458 - INFO -   + Reading nextgen-passing_2018.csv.gz


CPU times: user 34 ms, sys: 6.79 ms, total: 40.8 ms
Wall time: 42.3 ms


In [14]:
%%time
next_rec_df = load_files('nextgen-receiving')


2023-07-06 18:13:08,059 - INFO - Reading all files from nextgen-receiving
2023-07-06 18:13:08,061 - INFO -   + Reading nextgen-receiving_2021.csv.gz
2023-07-06 18:13:08,068 - INFO -   + Reading nextgen-receiving_2017.csv.gz
2023-07-06 18:13:08,074 - INFO -   + Reading nextgen-receiving_2019.csv.gz
2023-07-06 18:13:08,082 - INFO -   + Reading nextgen-receiving_2016.csv.gz
2023-07-06 18:13:08,090 - INFO -   + Reading nextgen-receiving_2020.csv.gz
2023-07-06 18:13:08,096 - INFO -   + Reading nextgen-receiving_2018.csv.gz
2023-07-06 18:13:08,103 - INFO -   + Reading nextgen-receiving_2022.csv.gz


CPU times: user 43.2 ms, sys: 8.46 ms, total: 51.7 ms
Wall time: 53.1 ms


In [15]:
%%time
next_rush_df = load_files('nextgen-rushing')

2023-07-06 18:13:08,736 - INFO - Reading all files from nextgen-rushing
2023-07-06 18:13:08,737 - INFO -   + Reading nextgen-rushing_2018.csv.gz
2023-07-06 18:13:08,741 - INFO -   + Reading nextgen-rushing_2022.csv.gz
2023-07-06 18:13:08,745 - INFO -   + Reading nextgen-rushing_2016.csv.gz
2023-07-06 18:13:08,749 - INFO -   + Reading nextgen-rushing_2020.csv.gz
2023-07-06 18:13:08,753 - INFO -   + Reading nextgen-rushing_2019.csv.gz
2023-07-06 18:13:08,757 - INFO -   + Reading nextgen-rushing_2021.csv.gz
2023-07-06 18:13:08,762 - INFO -   + Reading nextgen-rushing_2017.csv.gz


CPU times: user 25.6 ms, sys: 7.05 ms, total: 32.7 ms
Wall time: 32.6 ms


### <font color="#9370DB">players</font>

In [16]:
%%time
players_df = load_files('players')
players_df = transform_players(players_df)

2023-07-06 18:13:09,903 - INFO - Reading all files from players
2023-07-06 18:13:09,904 - INFO -   + Reading players.csv
2023-07-06 18:13:09,967 - INFO - Process players dataset...
2023-07-06 18:13:09,968 - INFO - drop players without gsis_ids - they won't link to player_stats
2023-07-06 18:13:09,981 - INFO - fill empty players status to 'NONE'
2023-07-06 18:13:09,985 - INFO - rename gsis_id to player_id...


CPU times: user 77.9 ms, sys: 7.05 ms, total: 84.9 ms
Wall time: 84.5 ms


---

# <font color=teal>store to database so we can perform some SQL operations</font>

In [17]:
def load_all_datasets_to_db(data: dict):
    data['schema'] = database_schema
    load_dims_to_db(data)


In [18]:
%%time
if LOAD_TO_DB:
    datasets.update({
        'players': players_df,
        'player_stats': stats_df,
        'adv_stats_def': advstats_def_df,
        'adv_stats_pass': advstats_pass_df,
        'adv_stats_rec': advstats_rec_df,
        'adv_stats_rush': advstats_rush_df,
        'nextgen_pass': next_pass_df,
        'nextgen_rec': next_rec_df,
        'nextgen_rush': next_rush_df
    })
    load_all_datasets_to_db(datasets)

2023-07-06 18:13:11,980 - INFO - create table play_actions in schema controls
2023-07-06 18:13:40,380 - INFO - create table game_drive in schema controls
2023-07-06 18:13:59,912 - INFO - create table play_analytics in schema controls
2023-07-06 18:15:12,896 - INFO - create table play_situations in schema controls
2023-07-06 18:15:41,620 - INFO - create table play_metrics in schema controls
2023-07-06 18:16:04,161 - INFO - create table player_events in schema controls
2023-07-06 18:16:13,347 - INFO - create table game_info in schema controls
2023-07-06 18:16:13,529 - INFO - create table player_participation in schema controls
2023-07-06 18:20:38,436 - INFO - create table players in schema controls
2023-07-06 18:20:39,942 - INFO - create table player_stats in schema controls
2023-07-06 18:20:56,542 - INFO - create table adv_stats_def in schema controls
2023-07-06 18:20:56,895 - INFO - create table adv_stats_pass in schema controls
2023-07-06 18:20:56,954 - INFO - create table adv_stats_r

CPU times: user 3min 41s, sys: 57.7 s, total: 4min 38s
Wall time: 7min 49s
